In [ ]:
!pip install pandas==2.0.3 numpy==1.25.2 scikit-learn==1.3.0 matplotlib==3.4.3 seaborn==0.11.2 xgboost==1.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 47.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 107.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 85.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 107.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 8.9 MB/s eta 0:00:00:00:0100:01
  Created wheel for matplotlib: filename=matplotlib-3.4.3-cp311-cp311-linux_x86_64.whl size=10441363 sha256=2db785b9539ca944920dfcea9aa7aedc6fff1c3c6150b14450db9f5dd9defc10
  Stored in directory: /root/.cache/pip/wheels/5a/99/fa/d6b0fc4a7c0f444b5f0ee8e7513f1d3095338bc1b458033e5d
Successfully built matplotlib
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Succe

# XGB 학습

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
import random as rn
RANDOM_SEED = 2025
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

def smape(gt, preds):
    gt= np.array(gt)
    preds = np.array(preds)
    v = 2 * abs(preds - gt) / (abs(preds) + abs(gt))
    score = np.mean(v) * 100
    return score

def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def custom_smape(preds, dtrain):
    labels = dtrain.get_label()
    return 'custom_smape', np.mean(2 * abs(preds - labels) / (abs(preds) + abs(labels))) * 100

train = pd.read_csv('/kaggle/input/dataset-ee/train.csv')
test = pd.read_csv('/kaggle/input/dataset-ee/test.csv')
building_info = pd.read_csv('/kaggle/input/dataset-ee/building_info.csv')

train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train.drop('num_date_time', axis = 1, inplace=True)

test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test.drop('num_date_time', axis = 1, inplace=True)

building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '학교': 'University',
    '백화점': 'Department Store',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    'IDC(전화국)': 'IDC',
    '호텔': 'Hotel'
}

building_info['building_type'] = building_info['building_type'].replace(translation_dict)

building_info['solar_power_utility'] = np.where(building_info.solar_power_capacity !='-',1,0)
building_info['ess_utility'] = np.where(building_info.ess_capacity !='-',1,0)

train = pd.merge(train, building_info, on='building_number', how='left')
test = pd.merge(test, building_info, on='building_number', how='left')

train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')
test['date_time'] = pd.to_datetime(test['date_time'], format='%Y%m%d %H')

# date time feature 생성
train['hour'] = train['date_time'].dt.hour
train['day'] = train['date_time'].dt.day
train['month'] = train['date_time'].dt.month
train['day_of_week'] = train['date_time'].dt.dayofweek #요일(월=0, 일=6)
# train['week'] = train.dt.isocalendar().week.astype(np.int32)
test['hour'] = test['date_time'].dt.hour
test['day'] = test['date_time'].dt.day
test['month'] = test['date_time'].dt.month
test['day_of_week'] = test['date_time'].dt.dayofweek #요일
# test['week'] = test.dt.isocalendar().week.astype(np.int32)

def calculate_day_values(dataframe, target_column, output_column, aggregation_func):
    result_dict = {}
    grouped_temp = dataframe.groupby(['building_number', 'month', 'day'])[target_column].agg(aggregation_func)
    for (building, month, day), value in grouped_temp.items():
        result_dict.setdefault(building, {}).setdefault(month, {})[day] = value
    dataframe[output_column] = [
        result_dict.get(row['building_number'], {}).get(row['month'], {}).get(row['day'], None)
        for _, row in dataframe.iterrows()
    ]

train['day_max_temperature'] = 0.0
train['day_mean_temperature'] = 0.0
calculate_day_values(train, 'temperature', 'day_max_temperature', 'max')
calculate_day_values(train, 'temperature', 'day_mean_temperature', 'mean')
calculate_day_values(train, 'temperature', 'day_min_temperature', 'min')
train['day_temperature_range'] = train['day_max_temperature'] - train['day_min_temperature']
calculate_day_values(test, 'temperature', 'day_max_temperature', 'max')
calculate_day_values(test, 'temperature', 'day_mean_temperature', 'mean')
calculate_day_values(test, 'temperature', 'day_min_temperature', 'min')
test['day_temperature_range'] = test['day_max_temperature'] - test['day_min_temperature']

######################## 특이값 제거 ########################
out_ind_li = []
# 1번
out_ind_1 = train.loc[(train['building_number'] ==1 ) & (train['power_consumption'] < 3000) ,:].index
out_ind_li += list(out_ind_1)
# 5번 (날짜 조건)
out_ind_5 = train.loc[
    (train['building_number'] == 5) &
    (train['date_time'].dt.strftime('%Y%m%d').astype(int) >= 20240801) &
    (train['date_time'].dt.strftime('%Y%m%d').astype(int) <= 20240815) &
    (train['power_consumption'] < 3200)
, :].index
out_ind_li += list(out_ind_5)
# 7번
out_ind_7 = train.loc[(train['building_number'] ==7 ) & (train['power_consumption'] < 3000) ,:].index
out_ind_li += list(out_ind_7)
# 12번
out_ind_12 = train.loc[(train['building_number'] ==12 ) & (train['power_consumption'] < 4000),:].index
out_ind_li += list(out_ind_12)
# 17번 (날짜 조건)
out_ind_17 = train.loc[
    (train['building_number'] == 17) &
    (train['date_time'].dt.strftime('%Y%m%d').astype(int) >= 20240619) &
    (train['date_time'].dt.strftime('%Y%m%d').astype(int) <= 20240701) &
    (train['power_consumption'] < 1500)
, :].index
out_ind_li += list(out_ind_17)
# 19번
out_ind_19 = train.loc[(train['building_number'] ==19 ) & (train['power_consumption'] > 3000) ,:].index
out_ind_li += list(out_ind_19)
# 26번
out_ind_26 = train.loc[(train['building_number'] ==26 ) & (train['power_consumption'] < 500) ,:].index
out_ind_li += list(out_ind_26)
# 30번
out_ind_30 = train.loc[(train['building_number'] ==30 ) & (train['power_consumption'] < 9000) ,:].index
out_ind_li += list(out_ind_30)
# 41번
out_ind_41 = train.loc[(train['building_number'] ==41 ) & (train['power_consumption'] < 2500),:].index
out_ind_li += list(out_ind_41)
# 42번
out_ind_42 = train.loc[(train['building_number'] ==42 ) & (train['power_consumption'] < 3000) ,:].index
out_ind_li+=list(out_ind_42)

# 44번 보류
out_ind_44 = train.loc[(train['building_number'] ==44 ) & (train['power_consumption'] < 900) ,:].index
out_ind_li+=list(out_ind_44)
# 52번 보류
out_ind_52 = train.loc[(train['building_number'] ==52 ) & (train['power_consumption'] < 4500) ,:].index
out_ind_li+=list(out_ind_52)

# 67번
out_ind_67 = train.loc[(train['building_number'] ==67 ) & (train['power_consumption'] < 8000) ,:].index
out_ind_li+=list(out_ind_67)
# 68번 보류
out_ind_68 = train.loc[(train['building_number'] ==68 ) & (train['power_consumption'] < 1000) ,:].index
out_ind_li+=list(out_ind_68)

# 72번
out_ind_72 = train.loc[
    (train['building_number'] == 72) &
    (train['date_time'].dt.strftime('%Y%m%d').astype(int) >= 20240601) &
    (train['date_time'].dt.strftime('%Y%m%d').astype(int) <= 20240615) &
    (train['power_consumption'] > 1830)
, :].index
out_ind_li += list(out_ind_72)

# 79번
out_ind_79 = train.loc[(train['building_number'] ==79 ) & (train['power_consumption'] < 1500) ,:].index
out_ind_li+=list(out_ind_79)
# 92번
out_ind_92 = train.loc[(train['building_number'] ==92 ) & (train['power_consumption'] < 200) ,:].index
out_ind_li+=list(out_ind_92)

# 실제 제거
print("특이값 제거 index 개수:", len(out_ind_li))
train = train.drop(index=out_ind_li).reset_index(drop=True)

outlier_idx = train.index[train['power_consumption'] == 0].tolist()
print("제로 power_consumption 제거할 행 개수:", len(outlier_idx))
print("인덱스 예시:", outlier_idx[:10])
train.drop(index=outlier_idx, inplace=True)
print("남은 행 개수:", train.shape[0])

# 휴일 지정 (IDC, 백화점 건물 제외) - 기본 규칙
holi_weekday = ['2024-06-06', '2024-08-15']

train['holiday'] = np.where(
    (~train['building_type'].isin(['IDC', 'Department Store'])) &
    ((train.day_of_week >= 5) | (train.date_time.dt.strftime('%Y-%m-%d').isin(holi_weekday))),
    1,
    0
)

test['holiday'] = np.where(
    (~test['building_type'].isin(['IDC', 'Department Store'])) &
    ((test.day_of_week >= 5) | (test.date_time.dt.strftime('%Y-%m-%d').isin(holi_weekday))),
    1,
    0
)

# >>> 추가 휴일 지정: 건물 27, 40, 59, 63에 대해 "매달 둘째 주 일요일"을 휴일로 처리
# - pandas의 day_of_week: 월=0, ..., 일=6
# - 둘째 주 일요일은 그 달의 날짜가 8~14 범위인 '일요일'
target_buildings = {27, 40, 59, 63}
# 27, 40, 59, 63
def mark_second_and_fourth_sunday_holiday(df):
    is_sunday = df['day_of_week'] == 6
    # 둘째 주 (8~14일), 넷째 주 (22~28일)
    is_2nd_week = df['day'].between(8, 14)
    is_4th_week = df['day'].between(22, 28)
    mask = df['building_number'].isin(target_buildings) & is_sunday & (is_2nd_week | is_4th_week)
    df.loc[mask, 'holiday'] = 1
    return df

train = mark_second_and_fourth_sunday_holiday(train)
test  = mark_second_and_fourth_sunday_holiday(test)
# >>> 32번 건물: 매달 둘째·넷째 월요일만 휴무. (기존 holiday 규칙 모두 덮어씀)
def apply_2nd_4th_monday_off_for_b32(df):
    is_b32 = df['building_number'] == 32

    # 우선 32번 건물은 전부 근무(0)로 초기화
    df.loc[is_b32, 'holiday'] = 0

    # 요일: 월=0, ..., 일=6
    is_monday = df['day_of_week'] == 0

    # 주차(week-of-month): 1~5
    wom = ((df['day'] - 1) // 7) + 1

    # 둘째/넷째 주 + 월요일인 곳만 휴무로 표시
    mask = is_b32 & is_monday & wom.isin([2, 4])
    df.loc[mask, 'holiday'] = 1
    return df

train = apply_2nd_4th_monday_off_for_b32(train)
test  = apply_2nd_4th_monday_off_for_b32(test)
# <<<
# >>> 2번 건물: 매주 토요일만 휴무(공휴일 근무). 기존 규칙을 모두 덮어씀.
def apply_sat_only_off(df):
    is_b2 = df['building_number'] == 2
    # 기본값: 전부 근무(holiday=0)
    df.loc[is_b2, 'holiday'] = 0
    # 토요일(월=0, ..., 토=5, 일=6)만 휴무
    df.loc[is_b2 & (df['day_of_week'] == 5), 'holiday'] = 1
    return df

train = apply_sat_only_off(train)
test  = apply_sat_only_off(test)

no_holiday_buildings = [58, 82, 97]

def clear_holiday_for_specific_buildings(df):
    mask = df['building_number'].isin(no_holiday_buildings)
    df.loc[mask, 'holiday'] = 0
    return df

train = clear_holiday_for_specific_buildings(train)
test  = clear_holiday_for_specific_buildings(test)
#시간
train['sin_hour'] = np.sin(2 * np.pi * train['hour']/23.0)
train['cos_hour'] = np.cos(2 * np.pi * train['hour']/23.0)
test['sin_hour'] = np.sin(2 * np.pi * test['hour']/23.0)
test['cos_hour'] = np.cos(2 * np.pi * test['hour']/23.0)

#날짜
train['sin_date'] = -np.sin(2 * np.pi * (train['month']+train['day']/31)/12)
train['cos_date'] = -np.cos(2 * np.pi * (train['month']+train['day']/31)/12)
test['sin_date'] = -np.sin(2 * np.pi * (test['month']+test['day']/31)/12)
test['cos_date'] = -np.cos(2 * np.pi * (test['month']+test['day']/31)/12)

#월
train['sin_month'] = -np.sin(2 * np.pi * train['month']/12.0)
train['cos_month'] = -np.cos(2 * np.pi * train['month']/12.0)
test['sin_month'] = -np.sin(2 * np.pi * test['month']/12.0)
test['cos_month'] = -np.cos(2 * np.pi * test['month']/12.0)

#요일
train['sin_dayofweek'] = -np.sin(2 * np.pi * (train['day_of_week']+1)/7.0)
train['cos_dayofweek'] = -np.cos(2 * np.pi * (train['day_of_week']+1)/7.0)
test['sin_dayofweek'] = -np.sin(2 * np.pi * (test['day_of_week']+1)/7.0)
test['cos_dayofweek'] = -np.cos(2 * np.pi * (test['day_of_week']+1)/7.0)

def CDH(xs):
    cumsum = np.cumsum(xs - 26)
    return np.concatenate((cumsum[:11], cumsum[11:] - cumsum[:-11]))

def calculate_and_add_cdh(dataframe):
    cdhs = []
    for i in range(1, 101):
        temp = dataframe[dataframe['building_number'] == i]['temperature'].values
        cdh = CDH(temp)
        cdhs.append(cdh)
    return np.concatenate(cdhs)

train['CDH'] = calculate_and_add_cdh(train)
test['CDH'] = calculate_and_add_cdh(test)
train['THI'] = 9/5*train['temperature'] - 0.55*(1-train['humidity']/100)*(9/5*train['humidity']-26)+32
test['THI'] = 9/5*test['temperature'] - 0.55*(1-test['humidity']/100)*(9/5*test['humidity']-26)+32
train['WCT'] = 13.12 + 0.6125*train['temperature'] - 11.37*(train['windspeed']**
                                                            0.16) + 0.3965*(train['windspeed']**0.16)*train['temperature']
test['WCT'] = 13.12 + 0.6125*test['temperature'] - 11.37*(test['windspeed']**
                                                            0.16) + 0.3965*(test['windspeed']**0.16)*test['temperature']

power_mean = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour', 'day_of_week'], aggfunc=np.mean).reset_index()
power_mean.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_mean']

power_std = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour', 'day_of_week'], aggfunc=np.std).reset_index()
power_std.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_std']

power_hour_mean = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour'], aggfunc=np.mean).reset_index()
power_hour_mean.columns = ['building_number', 'hour', 'hour_mean']

power_hour_std = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour'], aggfunc=np.std).reset_index()
power_hour_std.columns = ['building_number', 'hour', 'hour_std']

# Merge calculated features to 'train' and 'test' dataframes
train = train.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')

train = train.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')

train = train.merge(power_hour_mean, on=['building_number', 'hour'], how='left')
test = test.merge(power_hour_mean, on=['building_number', 'hour'], how='left')

train = train.merge(power_hour_std, on=['building_number', 'hour'], how='left')
test = test.merge(power_hour_std, on=['building_number', 'hour'], how='left')

train = train.reset_index(drop=True)

X = train.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity',
                'power_consumption','rainfall', 'sunshine', 'solar_radiation',
                'hour','day','month','day_of_week','date_time'],axis =1 )

Y = train[['building_type','power_consumption']]

test_X = test.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity','rainfall',
                   'hour','month','day_of_week','day','date_time'], axis=1)

type_list = []
for value in train.building_type.values:
    if value not in type_list:
        type_list.append(value)

from xgboost import XGBRegressor
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

RANDOM_SEED = 42
KFOLD_SPLITS = 7

def make_xgb():
    return XGBRegressor(
        learning_rate=0.05,
        n_estimators=5000,
        max_depth=10,
        subsample=0.7,
        colsample_bytree=0.5,
        min_child_weight=3,
        random_state=RANDOM_SEED,
        objective=weighted_mse(3),
        tree_method="gpu_hist",   # GPU 학습
        gpu_id=0,                 # GPU 번호 (Colab, Kaggle은 0)
        early_stopping_rounds=100
    )

def cv_fit_predict(model_fn, X_tr, y_tr, X_te):
    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_SEED)
    oof = np.zeros(len(y_tr), dtype=float)
    te_preds = []
    Xv = X_tr.values
    yv = y_tr.values
    for tr_idx, va_idx in kf.split(Xv):
        mdl = model_fn()
        X_tr_f, X_va_f = Xv[tr_idx], Xv[va_idx]
        y_tr_f, y_va_f = yv[tr_idx], yv[va_idx]
        mdl.fit(X_tr_f, np.log(y_tr_f), eval_set=[(X_va_f, np.log(y_va_f))], verbose=False)
        oof[va_idx] = np.exp(mdl.predict(X_va_f))
        te_preds.append(np.exp(mdl.predict(X_te.values)))
    te_mean = np.mean(te_preds, axis=0)
    return oof, te_mean

# === 1) GLOBAL 모델 ===
# X, Y, test_X 는 기존 코드에서 만든 것과 동일 (building_type, building_number 포함)
X_global  = X.drop(columns=["building_type"])  # building_type 더미 안씀(전역)
Xt_global = test_X.drop(columns=["building_type"])
oof_global, te_global = cv_fit_predict(make_xgb, X_global, Y["power_consumption"], Xt_global)

# === 2) TYPE 모델 ===
oof_type  = np.zeros(len(Y), dtype=float)
te_type   = np.zeros(len(test_X), dtype=float)
for t in X["building_type"].unique():
    idx_tr = (X["building_type"] == t)
    idx_te = (test_X["building_type"] == t)
    x_t  = X.loc[idx_tr].copy()
    xt_t = test_X.loc[idx_te].copy()
    # building_number 원핫 (유형 내부 features 정렬)
    x_t  = pd.get_dummies(x_t,  columns=["building_number"], drop_first=False)
    xt_t = pd.get_dummies(xt_t, columns=["building_number"], drop_first=False)
    xt_t = xt_t.reindex(columns=x_t.columns, fill_value=0)

    # drop type col
    x_t = x_t.drop(columns=["building_type"])
    xt_keep = xt_t.drop(columns=["building_type"])

    oof_t, te_t = cv_fit_predict(make_xgb, x_t, Y.loc[idx_tr, "power_consumption"], xt_keep)
    oof_type[idx_tr.values] = oof_t
    te_type[idx_te.values]  = te_t

# === 3) BUILDING 모델 ===
oof_bld = np.zeros(len(Y), dtype=float)
te_bld  = np.zeros(len(test_X), dtype=float)
min_rows = 200  # 표본이 너무 적은 건물은 스킵하고 상위 레벨에 맡김(원하면 조정)
for b in X["building_number"].unique():
    idx_tr = (X["building_number"] == b)
    idx_te = (test_X["building_number"] == b)
    if idx_tr.sum() < min_rows:
        # 표본 부족 → 상위 레벨값을 그대로 사용 (fallback)
        oof_bld[idx_tr.values] = oof_type[idx_tr.values]  # 또는 oof_global
        te_bld[idx_te.values]  = te_type[idx_te.values]   # 또는 te_global
        continue
    x_b  = X.loc[idx_tr].drop(columns=["building_type"])
    xt_b = test_X.loc[idx_te].drop(columns=["building_type"])
    oof_bb, te_bb = cv_fit_predict(make_xgb, x_b, Y.loc[idx_tr, "power_consumption"], xt_b)
    oof_bld[idx_tr.values] = oof_bb
    te_bld[idx_te.values]  = te_bb

# === 가중 블렌딩 ===
Wg, Wt, Wb = 0.2, 0.4, 0.4   # 시작 가중치 (튜닝 포인트)
oof_blend = Wg*oof_global + Wt*oof_type + Wb*oof_bld
te_blend  = Wg*te_global  + Wt*te_type  + Wb*te_bld

# 검증 SMAPE
print("Hier-Ensemble SMAPE:", smape(Y["power_consumption"].values, oof_blend))

# 제출
submit = pd.read_csv("/kaggle/input/dataset-ee/sample_submission.csv")
submit["answer"] = te_blend[submit.index]
submit.to_csv("hier_ensemble_xgb.csv", index=False)
print("Saved → hier_ensemble_xgb.csv")

# LGBM 학습

In [ ]:

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

# ---------- Utils ----------
def smape(gt, preds):
    gt = np.array(gt, dtype=float)
    preds = np.array(preds, dtype=float)
    v = 2 * np.abs(preds - gt) / (np.abs(preds) + np.abs(gt) + 1e-9)
    return float(np.mean(v) * 100)

def lgb_weighted_mse(alpha=3.0):
    """LightGBM custom objective: asymmetric MSE (under-prediction penalty > over-prediction)."""
    def _obj(y_true, y_pred):
        residual = (y_true - y_pred).astype(np.float64)
        # under-predict (residual>0) → larger gradient/hessian
        grad = np.where(residual > 0, -2.0*alpha*residual, -2.0*residual)
        hess = np.where(residual > 0,  2.0*alpha,          2.0)
        return grad, hess
    return _obj

# ---------- Load data (LOCAL PATHS) ----------
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
building_info = pd.read_csv('building_info.csv')

# ---------- Rename columns ----------
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
if 'num_date_time' in train.columns:
    train.drop('num_date_time', axis=1, inplace=True)

test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
if 'num_date_time' in test.columns:
    test.drop('num_date_time', axis=1, inplace=True)

building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '학교': 'University',
    '백화점': 'Department Store',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    'IDC(전화국)': 'IDC',
    '호텔': 'Hotel'
}
building_info['building_type'] = building_info['building_type'].replace(translation_dict)

# Feature flags for solar/ess presence
building_info['solar_power_utility'] = np.where(building_info['solar_power_capacity'] != '-', 1, 0)
building_info['ess_utility'] = np.where(building_info['ess_capacity'] != '-', 1, 0)

# Merge static info
train = pd.merge(train, building_info, on='building_number', how='left')
test  = pd.merge(test,  building_info, on='building_number', how='left')

# ---------- Datetime features ----------
train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')
test['date_time']  = pd.to_datetime(test['date_time'],  format='%Y%m%d %H')

for df in (train, test):
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['day_of_week'] = df['date_time'].dt.dayofweek

# ---------- Per-day aggregation features ----------
def calculate_day_values(dataframe, target_column, output_column, aggregation_func):
    grouped = dataframe.groupby(['building_number', 'month', 'day'])[target_column].agg(aggregation_func)
    result = {}
    for (b, m, d), v in grouped.items():
        result.setdefault(b, {}).setdefault(m, {})[d] = v
    dataframe[output_column] = [
        result.get(r['building_number'], {}).get(r['month'], {}).get(r['day'], np.nan)
        for _, r in dataframe.iterrows()
    ]

for df in (train, test):
    for out, func in [('day_max_temperature', 'max'),
                      ('day_mean_temperature', 'mean'),
                      ('day_min_temperature', 'min')]:
        calculate_day_values(df, 'temperature', out, func)
    df['day_temperature_range'] = df['day_max_temperature'] - df['day_min_temperature']

# ---------- Outlier removal (same rules as before) ----------
out_ind_li = []
out_ind_li += list(train.loc[(train['building_number']==1)  & (train['power_consumption'] < 3000)].index)
out_ind_li += list(train.loc[(train['building_number']==5)  &
                             (train['date_time'].dt.strftime('%Y%m%d').astype(int).between(20240801, 20240815)) &
                             (train['power_consumption'] < 3200)].index)
out_ind_li += list(train.loc[(train['building_number']==7)  & (train['power_consumption'] < 3000)].index)
out_ind_li += list(train.loc[(train['building_number']==12) & (train['power_consumption'] < 4000)].index)
out_ind_li += list(train.loc[(train['building_number']==17) &
                             (train['date_time'].dt.strftime('%Y%m%d').astype(int).between(20240619, 20240701)) &
                             (train['power_consumption'] < 1500)].index)
out_ind_li += list(train.loc[(train['building_number']==19) & (train['power_consumption'] > 3000)].index)
out_ind_li += list(train.loc[(train['building_number']==26) & (train['power_consumption'] < 500)].index)
out_ind_li += list(train.loc[(train['building_number']==30) & (train['power_consumption'] < 9000)].index)
out_ind_li += list(train.loc[(train['building_number']==41) & (train['power_consumption'] < 2500)].index)
out_ind_li += list(train.loc[(train['building_number']==42) & (train['power_consumption'] < 3000)].index)
out_ind_li += list(train.loc[(train['building_number']==44) & (train['power_consumption'] < 900)].index)
out_ind_li += list(train.loc[(train['building_number']==52) & (train['power_consumption'] < 4500)].index)
out_ind_li += list(train.loc[(train['building_number']==67) & (train['power_consumption'] < 8000)].index)
out_ind_li += list(train.loc[(train['building_number']==68) & (train['power_consumption'] < 1000)].index)
out_ind_li += list(train.loc[(train['building_number']==72) &
                             (train['date_time'].dt.strftime('%Y%m%d').astype(int).between(20240601, 20240615)) &
                             (train['power_consumption'] > 1830)].index)
out_ind_li += list(train.loc[(train['building_number']==79) & (train['power_consumption'] < 1500)].index)
out_ind_li += list(train.loc[(train['building_number']==92) & (train['power_consumption'] < 200)].index)

print("특이값 제거 index 개수:", len(out_ind_li))
train = train.drop(index=out_ind_li).reset_index(drop=True)

zero_idx = train.index[train['power_consumption'] == 0].tolist()
print("제로 power_consumption 제거할 행 개수:", len(zero_idx))
print("인덱스 예시:", zero_idx[:10])
train.drop(index=zero_idx, inplace=True)
print("남은 행 개수:", train.shape[0])

# ---------- Holiday features ----------
holi_weekday = ['2024-06-06', '2024-08-15']
train['holiday'] = np.where(
    (~train['building_type'].isin(['IDC', 'Department Store'])) &
    ((train['day_of_week'] >= 5) | (train['date_time'].dt.strftime('%Y-%m-%d').isin(holi_weekday))), 1, 0)
test['holiday'] = np.where(
    (~test['building_type'].isin(['IDC', 'Department Store'])) &
    ((test['day_of_week'] >= 5) | (test['date_time'].dt.strftime('%Y-%m-%d').isin(holi_weekday))), 1, 0)

target_buildings = {27, 40, 59, 63}
def mark_second_and_fourth_sunday_holiday(df):
    is_sunday = df['day_of_week'] == 6
    is_2nd = df['day'].between(8, 14)
    is_4th = df['day'].between(22, 28)
    df.loc[df['building_number'].isin(target_buildings) & is_sunday & (is_2nd | is_4th), 'holiday'] = 1
    return df

def apply_2nd_4th_monday_off_for_b32(df):
    is_b32 = df['building_number'] == 32
    df.loc[is_b32, 'holiday'] = 0
    is_monday = df['day_of_week'] == 0
    wom = ((df['day'] - 1) // 7) + 1
    df.loc[is_b32 & is_monday & wom.isin([2, 4]), 'holiday'] = 1
    return df

def apply_sat_only_off(df):
    is_b2 = df['building_number'] == 2
    df.loc[is_b2, 'holiday'] = 0
    df.loc[is_b2 & (df['day_of_week'] == 5), 'holiday'] = 1
    return df

def clear_holiday_for_specific_buildings(df):
    df.loc[df['building_number'].isin([58, 82, 97]), 'holiday'] = 0
    return df

for f in (mark_second_and_fourth_sunday_holiday,
          apply_2nd_4th_monday_off_for_b32,
          apply_sat_only_off,
          clear_holiday_for_specific_buildings):
    train = f(train)
    test  = f(test)

# ---------- Cyclic time features ----------
for df in (train, test):
    df['sin_hour'] = np.sin(2*np.pi*df['hour']/23.0)
    df['cos_hour'] = np.cos(2*np.pi*df['hour']/23.0)
    df['sin_date'] = -np.sin(2*np.pi*(df['month']+df['day']/31)/12)
    df['cos_date'] = -np.cos(2*np.pi*(df['month']+df['day']/31)/12)
    df['sin_month'] = -np.sin(2*np.pi*df['month']/12.0)
    df['cos_month'] = -np.cos(2*np.pi*df['month']/12.0)
    df['sin_dayofweek'] = -np.sin(2*np.pi*(df['day_of_week']+1)/7.0)
    df['cos_dayofweek'] = -np.cos(2*np.pi*(df['day_of_week']+1)/7.0)

# ---------- CDH / THI / WCT ----------
def CDH(xs):
    cumsum = np.cumsum(xs - 26)
    # 12-hr rolling cumulative by "windowed" trick
    return np.concatenate((cumsum[:11], cumsum[11:] - cumsum[:-11]))

def calculate_and_add_cdh(dataframe):
    cdhs = []
    for i in range(1, 101):
        temp = dataframe[dataframe['building_number'] == i]['temperature'].values
        cdhs.append(CDH(temp))
    return np.concatenate(cdhs)

train['CDH'] = calculate_and_add_cdh(train)
test['CDH']  = calculate_and_add_cdh(test)

train['THI'] = 9/5*train['temperature'] - 0.55*(1-train['humidity']/100)*(9/5*train['humidity']-26)+32
test['THI']  = 9/5*test['temperature']  - 0.55*(1-test['humidity']/100)*(9/5*test['humidity']-26)+32

train['WCT'] = 13.12 + 0.6125*train['temperature'] - 11.37*(train['windspeed']**0.16) + 0.3965*(train['windspeed']**0.16)*train['temperature']
test['WCT']  = 13.12 + 0.6125*test['temperature']  - 11.37*(test['windspeed']**0.16)  + 0.3965*(test['windspeed']**0.16)*test['temperature']

# ---------- Hourly stats features ----------
power_mean = pd.pivot_table(train, values='power_consumption',
                            index=['building_number','hour','day_of_week'], aggfunc=np.mean).reset_index()
power_mean.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_mean']

power_std  = pd.pivot_table(train, values='power_consumption',
                            index=['building_number','hour','day_of_week'], aggfunc=np.std).reset_index()
power_std.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_std']

hour_mean = pd.pivot_table(train, values='power_consumption',
                           index=['building_number','hour'], aggfunc=np.mean).reset_index()
hour_mean.columns = ['building_number', 'hour', 'hour_mean']

hour_std  = pd.pivot_table(train, values='power_consumption',
                           index=['building_number','hour'], aggfunc=np.std).reset_index()
hour_std.columns = ['building_number', 'hour', 'hour_std']

for df in (train, test):
    df.merge(power_mean, on=['building_number','hour','day_of_week'], how='left')
train = train.merge(power_mean, on=['building_number','hour','day_of_week'], how='left')
test  = test.merge(power_mean, on=['building_number','hour','day_of_week'], how='left')

train = train.merge(power_std, on=['building_number','hour','day_of_week'], how='left')
test  = test.merge(power_std, on=['building_number','hour','day_of_week'], how='left')

train = train.merge(hour_mean, on=['building_number','hour'], how='left')
test  = test.merge(hour_mean, on=['building_number','hour'], how='left')

train = train.merge(hour_std, on=['building_number','hour'], how='left')
test  = test.merge(hour_std, on=['building_number','hour'], how='left')

train = train.reset_index(drop=True)

# ---------- Final feature sets ----------
X = train.drop(columns=[
    'solar_power_capacity','ess_capacity','pcs_capacity',
    'power_consumption','rainfall','sunshine','solar_radiation',
    'hour','day','month','day_of_week','date_time'
], errors='ignore')

Y = train[['building_type','power_consumption']].copy()

test_X = test.drop(columns=[
    'solar_power_capacity','ess_capacity','pcs_capacity',
    'rainfall','hour','month','day_of_week','day','date_time'
], errors='ignore')

# ---------- LGBM builder ----------
RANDOM_SEED = 42
KFOLD_SPLITS = 7

def make_lgbm():
    return LGBMRegressor(
        n_estimators=5000,
        learning_rate=0.05,
        max_depth=10,
        subsample=0.7,
        colsample_bytree=0.5,
        min_child_samples=20,
        reg_lambda=0.0,
        random_state=RANDOM_SEED,
        objective=lgb_weighted_mse(alpha=3.0),  # custom asymmetric loss
        # device_type='gpu',  # GPU 사용 시 주석 해제 (환경에 따라)
    )

def cv_fit_predict(model_fn, X_tr, y_tr, X_te):
    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_SEED)
    oof = np.zeros(len(y_tr), dtype=float)
    te_preds = []
    Xv = X_tr.values
    yv = y_tr.values
    for tr_idx, va_idx in kf.split(Xv):
        mdl = model_fn()
        X_tr_f, X_va_f = Xv[tr_idx], Xv[va_idx]
        y_tr_f, y_va_f = yv[tr_idx], yv[va_idx]
        # log-transform target
        mdl.fit(
            X_tr_f, np.log(y_tr_f),
            eval_set=[(X_va_f, np.log(y_va_f))],
            eval_metric='l2',
            verbose=False,
            callbacks=[],
        )
        oof[va_idx] = np.exp(mdl.predict(X_va_f))
        te_preds.append(np.exp(mdl.predict(X_te.values)))
    te_mean = np.mean(te_preds, axis=0)
    return oof, te_mean

# ---------- 1) GLOBAL ----------
X_global  = X.drop(columns=['building_type'])
Xt_global = test_X.drop(columns=['building_type'])
oof_global, te_global = cv_fit_predict(make_lgbm, X_global, Y['power_consumption'], Xt_global)

# ---------- 2) TYPE ----------
oof_type = np.zeros(len(Y), dtype=float)
te_type  = np.zeros(len(test_X), dtype=float)
for t in X['building_type'].unique():
    idx_tr = (X['building_type'] == t)
    idx_te = (test_X['building_type'] == t)
    x_t  = X.loc[idx_tr].copy()
    xt_t = test_X.loc[idx_te].copy()

    # one-hot building_number inside each type block (align columns)
    x_t  = pd.get_dummies(x_t,  columns=['building_number'], drop_first=False)
    xt_t = pd.get_dummies(xt_t, columns=['building_number'], drop_first=False)
    xt_t = xt_t.reindex(columns=x_t.columns, fill_value=0)

    x_t = x_t.drop(columns=['building_type'])
    xt_keep = xt_t.drop(columns=['building_type'])

    oof_t, te_t = cv_fit_predict(make_lgbm, x_t, Y.loc[idx_tr, 'power_consumption'], xt_keep)
    oof_type[idx_tr.values] = oof_t
    te_type[idx_te.values]  = te_t

# ---------- 3) BUILDING ----------
oof_bld = np.zeros(len(Y), dtype=float)
te_bld  = np.zeros(len(test_X), dtype=float)
min_rows = 200  # 표본 적은 건물은 상위 레벨 예측 사용
for b in X['building_number'].unique():
    idx_tr = (X['building_number'] == b)
    idx_te = (test_X['building_number'] == b)
    if idx_tr.sum() < min_rows:
        oof_bld[idx_tr.values] = oof_type[idx_tr.values]
        te_bld[idx_te.values]  = te_type[idx_te.values]
        continue
    x_b  = X.loc[idx_tr].drop(columns=['building_type'])
    xt_b = test_X.loc[idx_te].drop(columns=['building_type'])
    oof_bb, te_bb = cv_fit_predict(make_lgbm, x_b, Y.loc[idx_tr, 'power_consumption'], xt_b)
    oof_bld[idx_tr.values] = oof_bb
    te_bld[idx_te.values]  = te_bb

# ---------- Blending ----------
Wg, Wt, Wb = 0.2, 0.4, 0.4
oof_blend = Wg*oof_global + Wt*oof_type + Wb*oof_bld
te_blend  = Wg*te_global  + Wt*te_type  + Wb*te_bld

print("Hier-Ensemble SMAPE:", smape(Y['power_consumption'].values, oof_blend))

# ---------- Save submission ----------
submit = pd.read_csv('sample_submission.csv')
submit['answer'] = te_blend[submit.index]
submit.to_csv('hier_ensemble_lgbm.csv', index=False)
print("Saved → hier_ensemble_lgbm.csv")

# Ensemble

In [ ]:
import pandas as pd
import numpy as np

# 파일 경로 설정
sub_xgb_path = "hier_ensemble_xgb.csv"          # XGB 결과
sub_lgbm_path = "hier_ensemble_lgbm.csv"    # LGBM 결과
sample_path = "sample_submission.csv"       # 샘플 제출
out_path = "ensemble_avg.csv"               # 최종 저장 경로

# 데이터 로드
sub_xgb = pd.read_csv(sub_xgb_path)
sub_lgbm = pd.read_csv(sub_lgbm_path)
sample = pd.read_csv(sample_path)

# 평균 앙상블 (answer 컬럼만 사용)
ensemble_answer = (sub_xgb["answer"].values + sub_lgbm["answer"].values) / 2

# 제출 파일 생성
submission = sample.copy()
submission["answer"] = ensemble_answer
submission.to_csv(out_path, index=False)
print(f"Saved → {out_path}")